In [522]:
import json

modulos = []

# Cargar el archivo JSON
with open('structure.json', 'r') as file:
    data = json.load(file)

# Recorrer todas las claves y valores del diccionario
for key, value in data['modulos'].items():
    modulos.append(key)
    print(f"Clave: {key}")

Clave: productos
Clave: categorias
Clave: almacenes


In [523]:
modulos

['productos', 'categorias', 'almacenes']

In [524]:
methods = ["get","post","put","delete"]

In [525]:
def gen_routes(value, modulo):
# Generación de rutas GET PUT DELETE POST
    if value.get("routes") == "auto":  # Si la clave 'routes' tiene el valor "auto"
        # Generación de routes
        with open(f"src/{modulo}/routes.py", 'w', encoding='utf-8') as file:
            header = f""" 
            from fastapi import APIRouter, Depends
            from .models import {modulo.capitalize()}
            from src.connect_db import SessionLocal
            from .schemas import {modulo.capitalize()}Base
            from sqlalchemy.orm import Session
            from src.auth.auth import get_current_user
            from sqlalchemy.inspection import inspect
            from typing import List, Dict
            router = APIRouter()
            
            # Dependencia para obtener la sesión de la base de datos
            def get_db():
                db = SessionLocal()
                try:
                    yield db
                finally:
                    db.close()
            
            """
            content = header 

            for method in methods:
                if(method == "get"):
                    path_route = f"""
            @router.{method}("/", response_model=list[{modulo.capitalize()}Base])"""
                    content_route = f"""
            def {method}_{modulo}(db: Session = Depends(get_db), current_user: dict = Depends(get_current_user)):
                {modulo} = db.query({modulo.capitalize()}).all()
                return {modulo}

            """
                    content += "\n".join([line.lstrip() for line in path_route.splitlines()]) + content_route
                    
                if(method == "post"):

                    path_route = f"""
            @router.{method}("/", response_model={modulo.capitalize()}Base)"""
                    content_route = f"""
            def {method}_{modulo}({modulo}: {modulo.capitalize()}Base, db: Session = Depends(get_db), current_user: dict = Depends(get_current_user)):
                {modulo} = {modulo.capitalize()}(
                    {
                        ", ".join([f"{keys.split(':')[0]}={modulo}.{keys.split(':')[0]}" for keys in value["model"]])
                    }
                    )
                db.add({modulo})
                db.commit()
                db.refresh({modulo})
                return {modulo}
                    """
                    content += "\n".join([line.lstrip() for line in path_route.splitlines()]) + content_route
            # Quitar una sola tabulación del principio de cada línea
            content = "\n".join([line[12:] if line.startswith(' ') else line for line in content.splitlines()])
            file.write(content)
    else:
         # Generación de routes
        with open(f"src/{key}/routes.py", 'w', encoding='utf-8') as file:
            content = """ 
            from fastapi import APIRouter, Depends, HTTPException, status
            from sqlalchemy.orm import Session
            router = APIRouter()

            @router.get("/")
            def read_root():
                return {"message": "Welcome to FastAPI!"}
            """
            # Quitar una sola tabulación del principio de cada línea
            content = "\n".join([line[12:] if line.startswith(' ') else line for line in content.splitlines()])
            file.write(content)
      

In [526]:
import os
def generate_services(value, modulo):
    # Generación de carpetas de los modulos
    directorio = f"src/{modulo}/services/__init__.py"
    os.makedirs(directorio, exist_ok=True)
    
    # Generación services
    if value.get("routes") == "auto":  # Si la clave 'routes' tiene el valor "auto"
        with open(f"src/{modulo}/services.py", 'w', encoding='utf-8') as file:
            header = """ 

            """

            content = header

            # Quitar una sola tabulación del principio de cada línea
            content = "\n".join([line[12:] if line.startswith(' ') else line for line in content.splitlines()])
            file.write(content)

In [527]:
def generate_models(value, modulo):
    if value.get("model")!=None:
        print(value.get("model"))
        with open(f"src/{key}/models.py", 'w', encoding='utf-8') as file:
                content = f"""
            from sqlalchemy import Column, Integer, String, Float
            from src.database import Base

            class {modulo.capitalize()}(Base):
                __tablename__ = '{modulo}'
                id = Column(Integer, primary_key=True, index=True)"""
                # Quitar una sola tabulación del principio de cada línea
                content = "\n".join([line[12:] if line.startswith(' ') else line for line in content.splitlines()])
                file.write(content)

        # Generación models
        for modelo in value.get("model", []):
            model_name, tipo = modelo.split(":")
            print(f"Nombre: {model_name}, Tipo: {tipo}")

            with open(f"src/{key}/models.py", 'a', encoding='utf-8') as file:
                if(tipo == "str"):
                    content = f"""
                {model_name} = Column(String)"""
                elif(tipo == "int"):
                    content = f"""
                {model_name} = Column(Integer)"""
                elif(tipo == "float"):
                    content = f"""
                {model_name} = Column(Float)"""
                # Quitar una sola tabulación del principio de cada línea
                content = "\n".join([line[12:] if line.startswith(' ') else line for line in content.splitlines()])
                file.write(content)


In [528]:
def generate_schemas(value, modulo):
    # Generación de schemas
    with open(f"src/{modulo}/schemas.py", 'w', encoding='utf-8') as file:
        content = f""" 
            from pydantic import BaseModel
            from typing import List

            class {modulo.capitalize()}Base(BaseModel):
        """
        for modelo in value.get("model", []):
            model_name, tipo = modelo.split(":")
            content = content + f"""
                {model_name}: {tipo}"""
        

        content = "\n".join([line[12:] if line.startswith(' ') else line for line in content.splitlines()])
        file.write(content)
        

In [529]:
import os
from pathlib import Path

# create src/__init__.py
src_init = Path('src/__init__.py')
src_init.touch()

for key, value in data['modulos'].items():
    modulo = key
    # Generación de carpetas de los modulos
    directorio = f"src/{key}"
    os.makedirs(directorio, exist_ok=True)

    # Generación de init de modulos
    file_init = Path(f"src/{key}/__init__.py")
    file_init.touch()
    os.makedirs(directorio, exist_ok=True)

    # Generación de rutas GET PUT DELETE POST
    gen_routes(value, modulo)

    # Generación models
    generate_models(value, modulo)

    # Generación schemas
    generate_schemas(value, modulo)
    
    # Generación services
    generate_services(value, modulo)
    



['name:str', 'stock:int', 'descripcion:str', 'price:float']
Nombre: name, Tipo: str
Nombre: stock, Tipo: int
Nombre: descripcion, Tipo: str
Nombre: price, Tipo: float
['name:str']
Nombre: name, Tipo: str
['name:str', 'stock:int', 'descripcion:str', 'price:float']
Nombre: name, Tipo: str
Nombre: stock, Tipo: int
Nombre: descripcion, Tipo: str
Nombre: price, Tipo: float
